# Cantilever Beam: Finite Element Method (FEM)
This tutorial demonstrates how to simulate a cantilever beam using the **Finite Element Method (FEM)** in SOFA. You will learn how to define a 3D grid topology, apply fixed constraints using a region of interest (ROI), and model elastic deformation with hexahedral elements.

### Learning Objectives
- Define a 3D structured grid using `RegularGridTopology`.
- Understand the role of `MechanicalObject` and `MeshMatrixMass` in 3D simulations.
- Use `BoxROI` to select indices for constraints.
- Configure a `StaticSolver` for equilibrium state simulation.
- Model linear elasticity using `HexahedronFEMForceField`.
- Visualize 3D mesh results using `k3d`.

---

## 1. Environment Setup
We begin by importing the core SOFA modules and initializing the runtime.

In [15]:
import Sofa
import SofaRuntime
SofaRuntime.init()

## 2. Creating the Scene Graph
We create the `root` node and define gravity. In this simulation, we use a standard gravity vector along the negative Y-axis.

In [16]:
root = Sofa.Core.Node("root")
root.gravity.value = [0.0, -9.81, 0.0]
root.dt.value = 0.01 # Simulation time step

## 3. Simulation Components

### Animation Loop and Solvers
For this simulation, we use a `StaticSolver`. Unlike dynamic solvers that integrate motion over time, a static solver finds the equilibrium state where internal elastic forces balance external forces (like gravity).

We also use:
- **NewtonRaphsonSolver**: A non-linear solver to find the equilibrium.
- **CGLinearSolver**: A Conjugate Gradient linear solver for the underlying system of equations.

In [17]:
root.addObject("DefaultAnimationLoop")

# ODE / Static Solvers
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Backward")
root.addObject("NewtonRaphsonSolver", 
               maxNbIterationsNewton=100, 
               maxNbIterationsLineSearch=10)

root.addObject("StaticSolver", name="solver")

# Linear Solver
SofaRuntime.importPlugin("Sofa.Component.LinearSolver.Iterative")
root.addObject("CGLinearSolver", iterations=25, tolerance=1e-5, threshold=1e-5)

## 4. Topology and State
A cantilever beam requires a geometric representation. We use a **structured grid** of hexahedral elements.

### Regular Grid Topology
`RegularGridTopology` creates a box-shaped grid defined by the number of samples (`nx`, `ny`, `nz`) and its spatial bounds.

In [18]:
SofaRuntime.importPlugin("Sofa.Component.Topology.Container.Grid")

# Define a beam elongated along the Z-axis
grid = root.addObject('RegularGridTopology', name="grid", 
                    nx=4, ny=4, nz=20, 
                    xmin=-9.0, xmax=-6.0, 
                    ymin=0.0, ymax=3.0, 
                    zmin=0.0, zmax=19.0)

### Mechanical State
The `MechanicalObject` stores the positions of the vertices (DOFs).

In [19]:
SofaRuntime.importPlugin("Sofa.Component.StateContainer")
state = root.addObject("MechanicalObject", template="Vec3", name="state")

### Mass
`MeshMatrixMass` computes the mass matrix based on the density and the volume of the elements. Here, we specify a `totalMass`.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Mass")
root.addObject("MeshMatrixMass", template="Vec3,Vec3", name="mass", totalMass=320.0)

[ERROR]  [MeshMatrixMass]  Requested template 'Vec3' cannot be found in the list of available templates [Vec1d,Vec1d, Vec1d,Vec2d, Vec1d,Vec3d, Vec2d,Vec2d, Vec2d,Vec3d, Vec3d,Vec3d]. Falling back to the first compatible template: 'Vec3d,Vec3d'.


ValueError: 4:Requested template 'Vec3' cannot be found in the list of available templates [Vec1d,Vec1d, Vec1d,Vec2d, Vec1d,Vec3d, Vec2d,Vec2d, Vec2d,Vec3d, Vec3d,Vec3d]. Falling back to the first compatible template: 'Vec3d,Vec3d'.


## 5. Constraints and Forces

### Fixed Constraint (BoxROI)
To create a *cantilever* beam, one end must be fixed. We use `BoxROI` (Region of Interest) to select all vertices within a specific volume, then pass those indices to a `FixedProjectiveConstraint`.

In [ ]:
# 1. Select the base of the beam (z near 0)
SofaRuntime.importPlugin("Sofa.Component.Engine.Select")
box = root.addObject('BoxROI', name="box", 
                    box=[-10.0, -1.0, -0.1,  -5.0, 4.0, 0.1], 
                    drawBoxes=True)

# 2. Fix the selected indices
SofaRuntime.importPlugin("Sofa.Component.Constraint.Projective")
root.addObject('FixedProjectiveConstraint', indices="@box.indices")

### Linear Elasticity (FEM)
The `HexahedronFEMForceField` computes the internal elastic forces of the beam based on the Finite Element Method.
- **Young Modulus**: Represents the stiffness of the material.
- **Poisson Ratio**: Represents the expansion/contraction of the material in directions perpendicular to the direction of compression.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.SolidMechanics.FEM.Elastic")
root.addObject('HexahedronFEMForceField', name="FEM", 
               youngModulus=20000.0, 
               poissonRatio=0.3, 
               method="large")

## 6. Running the Simulation
We initialize the scene and perform one or more steps. Since we use a `StaticSolver`, the simulation will attempt to reach the equilibrium position (the bent state) immediately.

In [ ]:
# Initialize the scene graph
Sofa.Simulation.initRoot(root)

Sofa.Simulation.animate(root, root.dt.value)

print("Simulation complete.")

## 7. Visualization
We use the `k3d` library to visualize the 3D mesh of the cantilever beam.

In [ ]:
import k3d

# Create the plot
plot = k3d.plot()

# Add the beam mesh to the plot
# Note: we use the triangles from the grid topology for visualization
plot += k3d.mesh(vertices=state.position.value, 
                 indices=grid.triangles.value,
                 opacity=0.8, 
                 color=0xE84E1B, 
                 name="Cantilever Beam")

plot.display()

### Summary and Analysis
In this tutorial, we simulated a 3D cantilever beam using FEM. 
- The **RegularGridTopology** provided the underlying structure.
- The **StaticSolver** allowed us to find the final resting state of the beam under gravity without calculating the oscillations.
- The **HexahedronFEMForceField** modeled the physical behavior of the material (stiffness and deformation).

The visualization shows the beam bending downwards, with the fixed end remaining stationary while the free end undergoes the maximum displacement.